# Connecting to an IBM Database Server in Python 

In [2]:
import ibm_db #Library allowing for database operations
import pandas as pd #Library for data analysis
import ibm_db_dbi

In [3]:
#Identitfy the database credentials
dsn_hostname = "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud" 
dsn_uid = "zyh88613"       
dsn_pwd = "wXi7njQU4nNmW2Z2"     

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"           
dsn_port = "32733"              
dsn_protocol = "TCPIP"           
dsn_security = "SSL"            

Now we need to create the database (Db2) connection!

In [4]:
#Create the connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud;PORT=32733;PROTOCOL=TCPIP;UID=zyh88613;PWD=wXi7njQU4nNmW2Z2;SECURITY=SSL;


In [5]:
#Create a connection to the database
try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )


Connected to database:  BLUDB as user:  zyh88613 on host:  54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud


In [6]:
#Retrieve metadata from the database Server and Client/Driver
server = ibm_db.server_info(conn)
client = ibm_db.client_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

print ("DRIVER_NAME:          ", client.DRIVER_NAME) 
print ("DRIVER_VER:           ", client.DRIVER_VER)
print ("DATA_SOURCE_NAME:     ", client.DATA_SOURCE_NAME)
print ("DRIVER_ODBC_VER:      ", client.DRIVER_ODBC_VER)
print ("ODBC_VER:             ", client.ODBC_VER)
print ("ODBC_SQL_CONFORMANCE: ", client.ODBC_SQL_CONFORMANCE)
print ("APPL_CODEPAGE:        ", client.APPL_CODEPAGE)
print ("CONN_CODEPAGE:        ", client.CONN_CODEPAGE)

DBMS_NAME:  DB2/LINUXX8664
DBMS_VER:   11.05.0800
DB_NAME:    BLUDB
DRIVER_NAME:           DB2CLI.DLL
DRIVER_VER:            11.05.0800
DATA_SOURCE_NAME:      BLUDB
DRIVER_ODBC_VER:       03.51
ODBC_VER:              03.01.0000
ODBC_SQL_CONFORMANCE:  EXTENDED
APPL_CODEPAGE:         1252
CONN_CODEPAGE:         1208


In [20]:
#Always remember to close the connection to avoid taking up resources!
ibm_db.close(conn)

True

### Create a Table in the Database (Db2)

In [8]:
#Drop the table INSTRUCTOR in case it exists
dropQuery = "drop table INSTRUCTOR"

#Execute the drop statement
dropStmt = ibm_db.exec_immediate(conn, dropQuery)

In [9]:
#Construct table
createquery= 'CREATE TABLE INSTRUCTOR(ID INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(20), LNAME VARCHAR(20), CITY VARCHAR(20), CCODE CHAR(2))'                      
#Execute the statement
creatstmt=ibm_db.exec_immediate(conn, createquery)

In [10]:
#Insert rows into table
insertquery1= "INSERT INTO INSTRUCTOR VALUES (1, 'Rav', 'Ahuja', 'TORONTO', 'CA')"
insertstmt1=ibm_db.exec_immediate(conn, insertquery1)
insertquery2="INSERT INTO INSTRUCTOR VALUES (2, 'Raul', 'Chong', 'Markham', 'CA'), (3,'Hima', 'Rasudevan', 'Chicago', 'US')"
insertstmt2=ibm_db.exec_immediate(conn, insertquery2)

In [11]:
#Retrieve data from INSTRUCTOR table
selectquery='SELECT * FROM INSTRUCTOR'
selectstmt=ibm_db.exec_immediate(conn, selectquery)

#Retrieve the first row of the dictionary
while ibm_db.fetch_both(selectstmt) != False:
    print('ID:', ibm_db.result(selectstmt,0), 'FNAME:', ibm_db.result(selectstmt, 'FNAME'))

ID: 1 FNAME: Rav
ID: 2 FNAME: Raul
ID: 3 FNAME: Hima


### Retrieve the database data

In [18]:
#Connection for pandas library
pconn = ibm_db_dbi.Connection(conn)

#Retrieve all rows from the INSTRUCTOR table and put in a data frame
df = pd.read_sql(selectquery, pconn)
df.head()

C:\Users\JoeDe\AppData\Local\Temp\ipykernel_22976\2672921617.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(selectquery, pconn)


,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,TORONTO,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Rasudevan,Chicago,US


In [22]:
#Close the connection
#ibm_db.close(conn)

## Resources
https://www.ibm.com/docs/en/db2/10.5?topic=db-connecting-database-server
